<a href="https://colab.research.google.com/github/jdmccarty/agent_exploration/blob/main/Kiteboard_Agentv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Kite Weather Agent

Works to recommend a kite size based on wind forecast at a specified location. Has access to the following tools:

*   Search
*   Math
*   Wolfram Alpha
*   Weather API

For additional notes or explanations, please see the Langchain Custom Agent with Tools Access which is the most similar template ([link](https://python.langchain.com/en/latest/modules/agents/agents/custom_agent_with_tool_retrieval.html?highlight=pirate))

#API Keys

In [ ]:
import os
#Replace 'HERE' with your API Keys
openai_api_key = os.environ['OPENAI_API_KEY'] = 'HERE'
openweather_api_key = os.environ["OPENWEATHERMAP_API_KEY"] = "HERE"
wolfram_api_key = os.environ["WOLFRAM_ALPHA_APPID"] = "HERE"
serpapi_api_key = os.environ["SERPAPI_API_KEY"] = "HERE"

#Install Libraries

In [ ]:
!pip install langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 907.7/907.7 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.9 MB/s eta 0:00:00


In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.4 MB/s eta 0:00:00


In [ ]:
!pip install pyowm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 39.2 MB/s eta 0:00:00


In [ ]:
!pip install wolframalpha

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install google-search-results

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32002 sha256=97172b6e6265cbcb88ba9044e52391150d7a23afc1dddc01337e1972183b88d9
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


#Load Agent and Tools

In [ ]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain, LLMMathChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re

from langchain.utilities import OpenWeatherMapAPIWrapper
from langchain.utilities.wolfram_alpha import WolframAlphaAPIWrapper
from langchain.utilities import SerpAPIWrapper

In [ ]:
# Define which tools the agent can use,
weather = OpenWeatherMapAPIWrapper()
wolfram = WolframAlphaAPIWrapper()
search = SerpAPIWrapper()

llm = OpenAI(temperature=0)

tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful when you need to answer questions about current events"
    ),
    Tool(
        name = "Wolfram Alpha",
        func=wolfram.run,
        description="useful for when you need to answer questions about Math, Science"
    ),
    Tool(
        name = "OpenWeatherMap",
        func=weather.run,
        description="Useful for fetching current weather information for a specified location"
    ),
    Tool(
        name = "Math",
        func=llm_math.run,
        description="Useful for basic math needs, formulas, and calculations"
    ),     
]

ALL_TOOLS = tools

#Tool Retreiver

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document

In [ ]:
docs = [Document(page_content=t.description, metadata={"index": i}) for i, t in enumerate(ALL_TOOLS)]

In [ ]:
vector_store = FAISS.from_documents(docs, OpenAIEmbeddings())


In [ ]:
retriever = vector_store.as_retriever()

def get_tools(query):
    docs = retriever.get_relevant_documents(query)
    return [ALL_TOOLS[d.metadata["index"]] for d in docs]

In [ ]:
#Tests that the tool retreival is working:
get_tools("What's the weather?")

[Tool(name='OpenWeatherMap', description='Useful for fetching current weather information for a specified location', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, func=<bound method OpenWeatherMapAPIWrapper.run of OpenWeatherMapAPIWrapper(owm=<pyowm.owm.OWM - API key=************************4fb9cc74, subscription type=free, PyOWM version=(3, 3, 0)>, openweathermap_api_key=None)>, coroutine=None),
 Tool(name='Search', description='useful when you need to answer questions about current events', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='f389b7f1fde87c517496bfbe332141ad2bbc6a5398be7848845683a8312f3f96', aiosession=None)>, coroutine=None),
 Tool(name='Wolfram Alpha', description='u

#Prompt Template

In [ ]:
# Set up the base template
template = """Answer the following questions as best you can as a polite expert assistant. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Do not include information you do not know.

Question: {input}
{agent_scratchpad}"""

In [ ]:
from typing import Callable
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    ############## NEW ######################
    # The list of tools available
    tools_getter: Callable
    
    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        ############## NEW ######################
        tools = self.tools_getter(kwargs["input"])
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in tools])
        return self.template.format(**kwargs)

In [ ]:
prompt = CustomPromptTemplate(
    template=template,
    tools_getter=get_tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

#Output Parser

In [ ]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [ ]:
output_parser = CustomOutputParser()

#Set up LLM, stop sequence, and the agent

In [ ]:
llm = OpenAI(temperature=0)
llm_math = LLMMathChain.from_llm(llm, verbose=True)

In [ ]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
tools = get_tools("What's the weather")
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

#Use the Agent

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [ ]:
weight = input("Please enter your weight in kilos (lbs/2.2): ")
city = input("Please enter the city: ")

agent_executor.run(f"What's the windspeed in {city}? \
Can you convert it to knots? \
Can you recommend a kite size using the fomula ({weight}*2.2175)/(windspeed in knots)\
Can you give the wind cardinal direction in N, NE, E, SE, S, SW, W, or NW? \
Can you round all answers to a single decimal place? Do not include fractions \
")

Please enter your weight in kilos (lbs/2.2): 100
Please enter the city: sf


> Entering new AgentExecutor chain...

Thought: I need to find the windspeed in SF and convert it to knots, then use the formula to recommend a kite size, and find the wind cardinal direction

Action: OpenWeatherMap
Action Input: San Francisco

Observation:In San Francisco, the current weather is as follows:
Detailed status: broken clouds
Wind speed: 6.71 m/s, direction: 248°
Humidity: 75%
Temperature: 
  - Current: 14.14°C
  - High: 18.62°C
  - Low: 12.12°C
  - Feels like: 13.57°C
Rain: {}
Heat index: None
Cloud cover: 75%
 I need to convert the wind speed to knots and round it to one decimal place
Action: Math
Action Input: 6.71 m/s to knots

Observation:Math is not a valid tool, try another one.
 I need to convert the wind speed to knots
Action: Wolfram Alpha
Action Input: 6.71 m/s to knots

Observation:Assumption: convert 6.71 m/s (meters per second) to knots 
Answer: 13.04 kn (knots)
 I need to use the fo

'The wind speed in SF is 13.0 kn, the recommended kite size is 17, and the wind cardinal direction is NNE.'